# Utility

- generate directories to help writing batch job commands

In [30]:
import os
from corrLib import readdata
import numpy as np
import matplotlib.pyplot as plt
from skimage import io
from shutil import copyfile

folder = '/home/zhengyang/data/DE/08132021'
l = readdata(folder, 'raw').sort_values(by='Dir')
for num, i in l.iterrows():
    print('python extractImages.py ' + os.path.split(i.Dir)[0] + ' &')

python extractImages.py /home/zhengyang/data/DE/08132021/2021-08-13_15h28m24s &
python extractImages.py /home/zhengyang/data/DE/08132021/2021-08-13_15h31m03s &
python extractImages.py /home/zhengyang/data/DE/08132021/2021-08-13_15h34m17s &
python extractImages.py /home/zhengyang/data/DE/08132021/2021-08-13_15h38m55s &
python extractImages.py /home/zhengyang/data/DE/08132021/2021-08-13_15h50m58s &
python extractImages.py /home/zhengyang/data/DE/08132021/2021-08-13_15h56m31s &
python extractImages.py /home/zhengyang/data/DE/08132021/2021-08-13_16h02m57s &
python extractImages.py /home/zhengyang/data/DE/08132021/2021-08-13_16h05m45s &
python extractImages.py /home/zhengyang/data/DE/08132021/2021-08-13_16h08m55s &
python extractImages.py /home/zhengyang/data/DE/08132021/2021-08-13_16h12m54s &
python extractImages.py /home/zhengyang/data/DE/08132021/2021-08-13_16h15m21s &
python extractImages.py /home/zhengyang/data/DE/08132021/2021-08-13_16h20m05s &
python extractImages.py /home/zhengyang/

In [12]:
i.Dir

'/home/zhengyang/data/DE/08132021/2021-08-13_16h35m25s/RawImage.raw'

In [17]:
# check if folder "images" exist
un_processed = []
for num, i in l.iterrows():
    d = os.path.join(os.path.split(i.Dir)[0], 'images')
    s = os.path.split(os.path.split(i.Dir)[0])[1]
    if os.path.exists(d) == True:
        print(s)
    else:
        un_processed.append(s)
if len(un_processed) != 0:
    print("-------unprocessed------")
    for s in un_processed:
        print(s)

2021-08-13_15h28m24s
2021-08-13_15h31m03s
2021-08-13_15h34m17s
2021-08-13_15h38m55s
2021-08-13_15h50m58s
2021-08-13_15h56m31s
2021-08-13_16h02m57s
2021-08-13_16h05m45s
2021-08-13_16h08m55s
2021-08-13_16h12m54s
2021-08-13_16h15m21s
2021-08-13_16h20m05s
2021-08-13_16h22m52s
2021-08-13_16h26m25s
2021-08-13_16h35m25s


## Extract track.txt, stagePosition.txt and reference image from raw data of bacteria.vi

- Folders to find: dd_tt_ref_n, dd_tt_n
- A separate log records the parameters in video n

In [2]:
folder = '/home/zhengyang/data/test_extractor'
sfL = next(os.walk(folder))[1]
sfL

['2021-07-21_16h41m52s_1', '2021-07-21_16h02m43s_ref_1']

**Procedure**
- Loop over all subfolders
- if folder contains "ref", read the number n after the last "_"
- create a folder with "n" as the name, in "main_folder/extract" if it does not exist already
- if ref: extract 1st image and save in extract_folder, otherwise: copy track.txt, stagePosition.txt and rawImageInfo.txt to extract_folder

In [36]:
folder = '/home/zhengyang/data/test_extractor'
sfL = next(os.walk(folder))[1]
for sf in sfL:
    if sf == 'extract':
        continue
    n = sf.split('_')[-1]
    extract_folder = os.path.join(folder, 'extract', n)
    img_folder = os.path.join(folder, sf)
    if os.path.exists(extract_folder) == False:
        os.makedirs(extract_folder)
        print(extract_folder)
    if 'ref' in sf.split('_'):
        # this is a reference image folder, only copy the 1st frame of .raw to extract_folder
        if check_necessary_files(img_folder):
            info_file = os.path.join(img_folder, 'RawImageInfo.txt')
            raw_file = os.path.join(img_folder, 'RawImage.raw')
            fps, h, w = read_raw_image_info(info_file)
            images = np.fromfile(raw_file, dtype='uint16')
            first_image = images[2:2+h*w].reshape(h, w)
            io.imsave(os.path.join(extract_folder, 'ref.tif'), first_image)
    else:
        info_file = os.path.join(img_folder, 'RawImageInfo.txt')
        SP_file = os.path.join(img_folder, 'StagePosition.txt')
        track_file = os.path.join(img_folder, 'Track', 'Track.txt')
        if os.path.exists(info_file) == True:
            copyfile(info_file, os.path.join(extract_folder, 'RawImageInfo.txt'))
        else:
            print("RawImageInfo.txt does not exist")
            
        if os.path.exists(SP_file) == True:
            copyfile(SP_file, os.path.join(extract_folder, 'StagePosition.txt'))
        else:
            print("StagePosition.txt does not exist")
        
        if os.path.exists(track_file) == True:
            copyfile(track_file, os.path.join(extract_folder, 'Track.txt'))
        else:
            print("Track.txt does not exist")

/home/zhengyang/data/test_extractor/extract/1


In [26]:
first_image

array([[11328, 10781, 11012, ..., 32767, 32185, 31406],
       [11169, 10619, 10420, ..., 29841, 28819, 28419],
       [10663, 10080, 10692, ..., 27203, 25943, 25107],
       ...,
       [19654, 19868, 20240, ...,  8869,  9066,  9066],
       [19764, 19106, 20448, ...,  9307,  9277,  9161],
       [18867, 19631, 19780, ...,  9320,  8980,  9216]], dtype=uint16)

In [10]:
def check_necessary_files(folder):
    """ Check for necessary files
    Check for RawImage.raw, RawImageInfo.txt (height, width, fps ...)
    
    Test:
    info_file = os.path.join(folder, 'RawImageInfo.txt')
    read_raw_image_info(info_file)
    """
    return os.path.exists(os.path.join(folder, 'RawImage.raw')) and \
            os.path.exists(os.path.join(folder, 'RawImageInfo.txt'))

def read_raw_image_info(info_file):
    """
    Read image info, such as fps and image dimensions, from RawImageInfo.txt
    """
    with open(info_file, 'r') as f:
        a = f.read()
    fps, h, w = a.split('\n')[0:3]
    return int(fps), int(h), int(w)

In [4]:
unique_nList = list(set(nList))
for n in unique_nList:
    

[1]